# ACTIVIDAD DE CLASIFICACIÓN DE TEXTO

En esta actividad vamos a trabajar en clasificar textos. Se recorrerá todo el proceso desde traer el dataset hasta proceder a dicha clasificación. Durante la actividad se llevarán a cabo muchos procesos como la creación de un vocabulario, el uso de embeddings y la creación de modelos.

Las cuestiones presentes en esta actividad están basadas en un Notebook creado por François Chollet, uno de los creadores de Keras y autor del libro "Deep Learning with Python". 

En este Notebook se trabaja con el dataset "Newsgroup20" que contiene aproximadamente 20000 mensajes que pertenecen a 20 categorías diferentes.

El objetivo es entender los conceptos que se trabajan y ser capaz de hacer pequeñas experimentaciones para mejorar el Notebook creado.

#Librerías

In [39]:
import numpy as np
import tensorflow as tf
from tensorflow import keras

# Descarga de Datos

In [40]:
data_path = keras.utils.get_file(
    "news20.tar.gz",
    "http://www.cs.cmu.edu/afs/cs.cmu.edu/project/theo-20/www/data/news20.tar.gz",
    untar=True,
)

In [41]:
import os
import pathlib

#Estructura de directorios del dataset
data_dir = pathlib.Path(data_path).parent / "20_newsgroup"
dirnames = os.listdir(data_dir)
print("Number of directories:", len(dirnames))
print("Directory names:", dirnames)

Number of directories: 20
Directory names: ['alt.atheism', 'comp.graphics', 'comp.os.ms-windows.misc', 'comp.sys.ibm.pc.hardware', 'comp.sys.mac.hardware', 'comp.windows.x', 'misc.forsale', 'rec.autos', 'rec.motorcycles', 'rec.sport.baseball', 'rec.sport.hockey', 'sci.crypt', 'sci.electronics', 'sci.med', 'sci.space', 'soc.religion.christian', 'talk.politics.guns', 'talk.politics.mideast', 'talk.politics.misc', 'talk.religion.misc']


In [42]:
#Algunos archivos de la categoria "com.graphics"
fnames = os.listdir(data_dir / "rec.autos")
print("Number of files in rec.autos:", len(fnames))
print("Some example filenames:", fnames[:5])

Number of files in rec.autos: 1000
Some example filenames: ['101551', '101552', '101553', '101554', '101555']


In [43]:
#Ejemplo de un texto de la categoría "com.graphics"
print(open(data_dir / "rec.autos" / "101551").read())

Path: cantaloupe.srv.cs.cmu.edu!das-news.harvard.edu!ogicse!uwm.edu!wupost!uunet!brunix!cs.brown.edu!cs012055
From: cs012055@cs.brown.edu (Hok-Chung Tsang)
Newsgroups: rec.autos
Subject: Re: Saturn's Pricing Policy
Message-ID: <1993Apr5.230808.581@cs.brown.edu>
Date: 5 Apr 93 23:08:08 GMT
Article-I.D.: cs.1993Apr5.230808.581
References: <C4oxwp.KKM@news.cso.uiuc.edu> <C4vIr5.L3r@shuksan.ds.boeing.com>
Sender: news@cs.brown.edu
Organization: Brown Computer Science Dept.
Lines: 51

In article <C4vIr5.L3r@shuksan.ds.boeing.com>, fredd@shuksan (Fred Dickey) writes:
|> CarolinaFan@uiuc (cka52397@uxa.cso.uiuc.edu) wrote:
|> : 	I have been active in defending Saturn lately on the net and would
|> : like to state my full opinion on the subject, rather than just reply to others'
|> : points.
|> : 	
|> : 	The biggest problem some people seem to be having is that Saturn
|> : Dealers make ~$2K on a car.  I think most will agree with me that the car is
|> : comparably priced with its competitors, t

In [44]:
#Algunos archivos de la categoria "talk.politics.misc"
fnames = os.listdir(data_dir / "talk.politics.misc")
print("Number of files in talk.politics.misc:", len(fnames))
print("Some example filenames:", fnames[:5])

Number of files in talk.politics.misc: 1000
Some example filenames: ['124146', '176845', '176846', '176847', '176849']


In [45]:
#Ejemplo de un texto de la categoría "talk.politics.misc"
print(open(data_dir / "talk.politics.misc" / "178463").read())

Xref: cantaloupe.srv.cs.cmu.edu talk.politics.guns:54219 talk.politics.misc:178463
Newsgroups: talk.politics.guns,talk.politics.misc
Path: cantaloupe.srv.cs.cmu.edu!magnesium.club.cc.cmu.edu!news.sei.cmu.edu!cis.ohio-state.edu!magnus.acs.ohio-state.edu!usenet.ins.cwru.edu!agate!spool.mu.edu!darwin.sura.net!martha.utcc.utk.edu!FRANKENSTEIN.CE.UTK.EDU!VEAL
From: VEAL@utkvm1.utk.edu (David Veal)
Subject: Re: Proof of the Viability of Gun Control
Message-ID: <VEAL.749.735192116@utkvm1.utk.edu>
Lines: 21
Sender: usenet@martha.utcc.utk.edu (USENET News System)
Organization: University of Tennessee Division of Continuing Education
References: <1qpbqd$ntl@access.digex.net> <C5otvp.ItL@magpie.linknet.com>
Date: Mon, 19 Apr 1993 04:01:56 GMT

[alt.drugs and alt.conspiracy removed from newsgroups line.]

In article <C5otvp.ItL@magpie.linknet.com> neal@magpie.linknet.com (Neal) writes:

>   Once the National Guard has been called into federal service,
>it is under the command of the present. Tha N

In [58]:
samples = []
labels = []
class_names = []
class_index = 0
for dirname in sorted(os.listdir(data_dir)):
    class_names.append(dirname)
    dirpath = data_dir / dirname
    fnames = os.listdir(dirpath)
    print("Processing %s, %d files found" % (dirname, len(fnames)))
    for fname in fnames:
        fpath = dirpath / fname
        f = open(fpath, encoding="latin-1")
        content = f.read()
        lines = content.split("\n")
        lines = lines[10:]
        content = "\n".join(lines)
        samples.append(content)
        labels.append(class_index)
    class_index += 1

print("Classes:", class_names)
print("Number of samples:", len(samples))

Processing alt.atheism, 1000 files found
Processing comp.graphics, 1000 files found
Processing comp.os.ms-windows.misc, 1000 files found
Processing comp.sys.ibm.pc.hardware, 1000 files found
Processing comp.sys.mac.hardware, 1000 files found
Processing comp.windows.x, 1000 files found
Processing misc.forsale, 1000 files found
Processing rec.autos, 1000 files found
Processing rec.motorcycles, 1000 files found
Processing rec.sport.baseball, 1000 files found
Processing rec.sport.hockey, 1000 files found
Processing sci.crypt, 1000 files found
Processing sci.electronics, 1000 files found
Processing sci.med, 1000 files found
Processing sci.space, 1000 files found
Processing soc.religion.christian, 997 files found
Processing talk.politics.guns, 1000 files found
Processing talk.politics.mideast, 1000 files found
Processing talk.politics.misc, 1000 files found
Processing talk.religion.misc, 1000 files found
Classes: ['alt.atheism', 'comp.graphics', 'comp.os.ms-windows.misc', 'comp.sys.ibm.pc.ha

# Mezclando los datos para separarlos en Traning y Test

In [80]:
# Shuffle the data
seed = 1337
rng = np.random.RandomState(seed)
rng.shuffle(samples)
rng = np.random.RandomState(seed)
rng.shuffle(labels)

# Extract a training & validation split
validation_split = 0.2
num_validation_samples = int(validation_split * len(samples))
train_samples = samples[:-num_validation_samples]
val_samples = samples[-num_validation_samples:]
train_labels = labels[:-num_validation_samples]
val_labels = labels[-num_validation_samples:]

¿Por qué mezclamos los datos antes de separarlos en entrenamiento y validación?

In [81]:
#Tu respuesta aqui

#Es necesario dividirlas para asegurarnos que las clases estén correctamente balanceadas tanto en la distribución que se usará para entrenamiento como para la de validación. 

# Tokenización de las palabras con TextVectorization 

In [82]:
from tensorflow.keras.layers import TextVectorization
vectorizer = TextVectorization(max_tokens=20000, output_sequence_length=200)
text_ds = tf.data.Dataset.from_tensor_slices(train_samples).batch(128)
vectorizer.adapt(text_ds)

In [83]:
vectorizer.get_vocabulary()[:5]

['', '[UNK]', 'the', 'to', 'of']

In [84]:
len(vectorizer.get_vocabulary())

20000

Pregunta. En la construcción del vocabulario hemos limitado el número de tokens ¿Podrías indicar el número de token diferentes o tamaño del vocabulario sin limitar el número de tokens? Es decir, ¿Cuántas palabras diferentes existen en los documentos procesados como instancias?

In [85]:
#Tu código aqui

#Para responder a la pregunta hay que hacer uso nuevo del método TextVectorization pero en esta ocasión no limitarle el número de tokens, es decir, eliminando donde indica max_tokens=20.000. A continuación se deja el código para resolver esta pregunta.
vectorizer2 = TextVectorization(output_sequence_length=200)
text_ds = tf.data.Dataset.from_tensor_slices(train_samples).batch(128)
vectorizer2.adapt(text_ds)


In [86]:
vectorizer2.get_vocabulary()[:5]

UnicodeDecodeError: 'utf-8' codec can't decode byte 0xc2 in position 6: unexpected end of data

# Viendo la salida de Vectorizer

In [87]:
output = vectorizer([["the cat sat on the mat"]])
output.numpy()[0, :6]

array([   2, 3644, 1711,   15,    2, 6427], dtype=int64)

In [76]:
output

<tf.Tensor: shape=(1, 200), dtype=int64, numpy=
array([[   2, 3456, 1682,   15,    2, 5776,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,   

In [89]:
voc = vectorizer.get_vocabulary()
word_index = dict(zip(voc, range(len(voc))))

In [90]:
test = ["the", "cat", "sat", "on", "the", "mat"]
[word_index[w] for w in test]

[2, 3644, 1711, 15, 2, 6427]

Pregunta. La salida de vectorizer() para codificar los tokens ["El", "gato", "está", "sobre", "el", "tejado"] es la siguiente [1, 121, 405, 1, 45, 4561]. Si cada uno de los valores indica el índice en el que se encuentra cada palabra en el array creado para codificarla. ¿Podría ser correcta esta salida?

In [19]:
#Tu respuesta aqui

#No es correcta la salida porque como se puede ver el token “El” ha sido codificado con un 2 (es decir, con todo ceros salvo la un 1 en la posición 2) y el token “sobre” también ha sido codificado con un 2, es decir, exactamente igual que el token “El”. Cada “token” debe tener una codificación diferente. Para más error, se puede ver que “El” posteriormente tiene asignada otra codificación diferente 45 (es decir, un vector con todo ceros salvo un uno en la posición 45. Esto no es posible. 

# Tokenización de los datos de entrenamiento y validación

In [91]:
x_train = vectorizer(np.array([[s] for s in train_samples])).numpy()
x_val = vectorizer(np.array([[s] for s in val_samples])).numpy()

y_train = np.array(train_labels)
y_val = np.array(val_labels)

#Creación del modelo con un embedding hecho a mano con redes neuronales clásicas


In [92]:
modeloEmbeddingManual = keras.models.Sequential()
modeloEmbeddingManual.add(keras.layers.Embedding(20000, 10, input_length=200))
modeloEmbeddingManual.add(keras.layers.Flatten())
modeloEmbeddingManual.add(keras.layers.Dense(512, activation='relu'))
modeloEmbeddingManual.add(keras.layers.Dropout(0.3))
modeloEmbeddingManual.add(keras.layers.Dense(20, activation='softmax'))

In [93]:
modeloEmbeddingManual.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
modeloEmbeddingManual.compile(loss="sparse_categorical_crossentropy", optimizer="rmsprop", metrics=["acc"])
modeloEmbeddingManual.fit(x_train, y_train, batch_size=128, epochs=20, validation_data=(x_val, y_val))
print(modeloEmbeddingManual.summary())

Epoch 1/20
125/125 [==============================] - 2s 5ms/step - loss: 2.7250 - acc: 0.1313 - val_loss: 2.3082 - val_acc: 0.2331
Epoch 2/20
125/125 [==============================] - 0s 4ms/step - loss: 1.9168 - acc: 0.3764 - val_loss: 1.7515 - val_acc: 0.4106
Epoch 3/20
125/125 [==============================] - 1s 4ms/step - loss: 1.2874 - acc: 0.6108 - val_loss: 1.3574 - val_acc: 0.5459
Epoch 4/20
125/125 [==============================] - 1s 4ms/step - loss: 0.8173 - acc: 0.7655 - val_loss: 1.1467 - val_acc: 0.6199
Epoch 5/20
125/125 [==============================] - 0s 4ms/step - loss: 0.5309 - acc: 0.8477 - val_loss: 1.0378 - val_acc: 0.6532
Epoch 6/20
125/125 [==============================] - 0s 4ms/step - loss: 0.3659 - acc: 0.8987 - val_loss: 1.0345 - val_acc: 0.6612
Epoch 7/20
125/125 [==============================] - 1s 4ms/step - loss: 0.2644 - acc: 0.9246 - val_loss: 1.0548 - val_acc: 0.6799
Epoch 8/20
125/125 [==============================] - 0s 4ms/step - loss: 0.

#Evaluación

In [94]:
string_input = keras.Input(shape=(1,), dtype="string")
x = vectorizer(string_input)
preds = modeloEmbeddingManual(x)
end_to_end_model = keras.Model(string_input, preds)

probabilities = end_to_end_model.predict(
    [["this message is about computer graphics and 3D modeling"]]
)

class_names[np.argmax(probabilities[0])]

1/1 [==============================] - 0s 105ms/step


'comp.graphics'

In [95]:
probabilities = end_to_end_model.predict(
    [["we are talking about politics"]]
)

class_names[np.argmax(probabilities[0])]

1/1 [==============================] - 0s 37ms/step


'misc.forsale'

In [96]:
probabilities = end_to_end_model.predict(
    [["we are talking about religion"]]
)

class_names[np.argmax(probabilities[0])]

1/1 [==============================] - 0s 36ms/step


'misc.forsale'